In [4]:
import pandas as pd
from opensky_api import OpenSkyApi

# Criar conexão com a API
api = OpenSkyApi()

# Obter estados das aeronaves
states = api.get_states()

# Converter em lista de dicionários para facilitar
data = []
for s in states.states:
    data.append({
        "icao24": s.icao24,
        "callsign": s.callsign,
        "origin_country": s.origin_country,
        "time_position": s.time_position,
        "last_contact": s.last_contact,
        "longitude": s.longitude,
        "latitude": s.latitude,
        "baro_altitude": s.baro_altitude,
        "on_ground": s.on_ground,
        "velocity": s.velocity,
        "true_track": s.true_track,
        "vertical_rate": s.vertical_rate
    })

# Criar DataFrame
df = pd.DataFrame(data)

print(df.head())


   icao24  callsign origin_country  time_position  last_contact  longitude  \
0  ae1fa7            United States   1.756420e+09    1756420246  -116.7198   
1  ab1644  UAL2116   United States   1.756420e+09    1756420249   -82.0330   
2  80162c  AXB252            India   1.756420e+09    1756420247    56.8494   
3  aa9321  UAL57     United States   1.756420e+09    1756420242   -74.1685   
4  ab6fdd  AAL1751   United States   1.756420e+09    1756420249   -77.9467   

   latitude  baro_altitude  on_ground  velocity  true_track  vertical_rate  
0   43.4592        1074.42      False     61.72      111.00           0.65  
1   38.1883       10965.18      False    200.41      242.65          -0.33  
2   25.0857        7216.14      False    227.19      112.50           5.85  
3   40.7009            NaN       True      0.00       95.62            NaN  
4   32.5338        9144.00      False    239.86      195.17           0.00  
